In [ ]:
# imports

import json
import requests
import pandas as pd
import os

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [ ]:
#yelp_foursquare_EDA.ipynb notebook 2- Questions in steps.

##Question-1: Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice.

fsq_api_key = os.environ["fsq_api_key"]

foursqr_headers = {
    "accept": "application/json",
    "Authorization": fsq_api_key
}

fsq_data = []

for each_station in new_stations:
    url = f"https://api.foursquare.com/v3/places/search?ll={each_station['latitude']},{each_station['longitude']}&radius=100000&categories=10000%2C13000&fields=geocodes%2Cname%2Clocation%2Ccategories%2Cdistance%2Crating"
    response = requests.get(url, headers=foursqr_headers)
    fsq_results = response.json()["results"]

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
## Question-2: Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc).

    all_fsq_pois = list()
    for each_poi in fsq_results:
      categories = list()
      for category in each_poi['categories']:
        categories.append(category['name'])
      poi_data =  {"POI Name": each_poi['name'], "Full POI Location": each_poi['location'], "Categories" : categories, "Rating": each_poi['rating'],  "Coordinates": each_poi['geocodes']['main'],  "Distance from station": each_poi['distance']}   
      all_fsq_pois.append(poi_data)
        
    ## Convert to dataframe for later use:
    df_fsq_pois = pd.DataFrame(all_fsq_pois)    
        
    ## Extract the "Distance from station" values from all_fsq_pois
    distances = [poi["Distance from station"] for poi in all_fsq_pois]

    ## Find the lowest integer value
    lowest_distance = min(distances)

    ## Create final fsq data file
    fsq_data.append({"Station Name": each_station["Station Name"], "latitude": each_station["latitude"], "longitude": each_station["longitude"], "Num_Bikes": each_station["Num_Bikes"], "Empty_slots": each_station["Empty_slots"], 'Foursquare POIS data': all_fsq_pois, "Shortest FSQ Distance from Station": lowest_distance})

Put your parsed results into a DataFrame

In [ ]:
## Question-3: Put your parsed results into a DataFrame.

df_all_fsq_pois = pd.DataFrame(fsq_data)  
print('THIS IS FOURSQUARE DATA EXTRACTED OUT ON BASIS OF CITYBIK DATA') 
print(df_all_fsq_pois)

## Convert dataframe to csv for better understanding of data.
df_all_fsq_pois.to_csv('fsqdata.csv', index=False)

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [ ]:
## Question-4: Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice.

yelp_api_key = os.environ["yelp_api_key"]

yelp_header = {"Authorization": yelp_api_key,
              "accept": "application/json"}

yelp_data = []

for each_station in new_stations:
    url = f"https://api.yelp.com/v3/businesses/search?latitude={each_station['latitude']}&longitude={each_station['longitude']}&radius=1000&categories=restaurant&categories=bar&categories=entertainment&sort_by=rating&limit=20"
    response = requests.get(url, headers=yelp_header)
    yelp_results = response.json()["businesses"]

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
## Question-5: Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc).
    all_yelp_pois = list()
    for each_poi in yelp_results:
      categories = list()
      for category in each_poi['categories']:
        categories.append(category['title'])
      poi_data =  {"Review_count": each_poi['review_count'], "POI Name": each_poi['name'], "Full POI Location": each_poi['location'], "Categories" : categories, "Rating": each_poi['rating'], "Coordinates": each_poi['coordinates'], "Distance from station": each_poi['distance'], "Is_business_closed": each_poi['is_closed']}   
      all_yelp_pois.append(poi_data)
      
    ##Convert to dataframe for later use:
    df_yelp_pois = pd.DataFrame(all_yelp_pois)
    
    ##Extract the "Distance from station" values from all_fsq_pois
    distances = [poi["Distance from station"] for poi in all_yelp_pois]

    ##Find the lowest integer value
    lowest_distance = min(distances)

    ##Add the lowest_distance to fsq_data
    yelp_data.append({"Station Name": each_station["Station Name"], "latitude": each_station["latitude"], "longitude": each_station["longitude"], "Num_Bikes": each_station["Num_Bikes"], "Empty_slots": each_station["Empty_slots"], 'Yelp POIS data': all_yelp_pois, "Shortest Yelp Distance from Station": lowest_distance})

Put your parsed results into a DataFrame

In [ ]:
## Question-6: Put your parsed results into a DataFrame.
df_all_yelp_pois = pd.DataFrame(yelp_data)  
print('THIS IS YELP DATA EXTRACTED OUT ON BASIS OF CITYBIK DATA') 
print(df_all_yelp_pois)

## Convert dataframe to csv for better understanding of data.
df_all_yelp_pois.to_csv('yelpdata.csv', index=False)

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

## Question-7: Which API provided you with more complete data? Provide an explanation.
# Clearly Yelp is better based on on the more rich information shown in above output, for example it is also telling me whether a POI is closed or not.
# Review count is another additional key in Yelp compared to FSQ.

Get the top 10 restaurants according to their rating

In [ ]:
## Question-8: Get the top 10 restaurants according to their rating.

### top 10 FourSquare restaurants

fsq_top_10_ratings = df_fsq_pois.sort_values(by="Rating", ascending=False).head(10)
print('FSQ top 10')
print(fsq_top_10_ratings[["Rating", "POI Name"]])

### top 10 Yelp restaurants

yelp_top_10_ratings = df_yelp_pois.sort_values(by="Rating", ascending=False).head(10)
print('Yelp top 10')
print(yelp_top_10_ratings[["Rating", "POI Name"]])